## Basic Generation Training

In [ ]:
%cd ../MedSinGAN

In [ ]:
# Run a best combination for a given image
! python main_train.py --train_mode generation --input_name ../pipeline_data_tcia/benign/benign.png --train_stages 16 --niter 2500 --gpu 0 --train_depth 9

## Combination Generation Testing

In [ ]:
# Import os to run experiments
import os

# Parameters to vary in generation mode
gen_test_stages = [6,8,10,12]
gen_test_lr = [0.1, 0.5]
gen_test_niter = [500, 1000, 2000]

In [ ]:
# Test all parameters to see improvements
for elem in gen_test_stages:
  os.system('python main_train.py --train_mode generation --input_name ../images/benign.png --train_stages ' + str(elem) + ' --gpu 0\n')
  
for elem in gen_test_lr:
  os.system('python main_train.py --train_mode generation --input_name ../images/benign.png --lr_scale ' + str(elem) + ' --gpu 0\n')

for elem in gen_test_niter:
  os.system('python main_train.py --train_mode generation --input_name ../images/benign.png --niter ' + str(elem) + ' --gpu 0\n')

## Save Model

In [ ]:
# Zip everything from this model
! zip -r current_model.zip .

## Delete Current Model

# Delete everything that the model produces
! rm -r mlruns TrainedModels